# Metadata

> Metadata introspection for the Voxtral VLLM plugin used by cjm-ctl to generate the registration manifest.

In [ ]:
#| default_exp meta

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import sys
from typing import Any, Dict
from cjm_transcription_plugin_voxtral_vllm import __version__

In [ ]:
#| export
def get_plugin_metadata() -> Dict[str, Any]: # Plugin metadata for manifest generation
    """Return metadata required to register this plugin with the PluginManager."""
    # Fallback base path (current behavior for backward compatibility)
    base_path = os.path.dirname(os.path.dirname(sys.executable))
    
    # Use CJM config if available, else fallback to env-relative paths
    cjm_data_dir = os.environ.get("CJM_DATA_DIR")
    cjm_models_dir = os.environ.get("CJM_MODELS_DIR")
    
    # Plugin data directory
    plugin_name = "cjm-transcription-plugin-voxtral-vllm"
    if cjm_data_dir:
        data_dir = os.path.join(cjm_data_dir, plugin_name)
    else:
        data_dir = os.path.join(base_path, "data")
    
    db_path = os.path.join(data_dir, "voxtral_vllm_transcriptions.db")
    
    # Ensure data directory exists
    os.makedirs(data_dir, exist_ok=True)
    
    # HuggingFace cache: use models_dir if configured
    if cjm_models_dir:
        hf_home = os.path.join(cjm_models_dir, "huggingface")
    else:
        hf_home = os.path.join(base_path, ".cache", "huggingface")

    return {
        "name": plugin_name,
        "version": __version__,
        "type": "transcription",
        "category": "transcription",
        "interface": "cjm_transcription_plugin_system.plugin_interface.TranscriptionPlugin",
        
        "module": "cjm_transcription_plugin_voxtral_vllm.plugin",
        "class": "VoxtralVLLMPlugin",
        
        # Critical: The absolute path to THIS environment's python
        "python_path": sys.executable,
        
        "db_path": db_path,
        
        # Voxtral via vLLM requires significant GPU resources
        # Mini: ~8GB VRAM, Small: ~48GB VRAM
        "resources": {
            "requires_gpu": True,
            "min_gpu_vram_mb": 8192,
            "recommended_gpu_vram_mb": 16384,
            "min_system_ram_mb": 16384
        },
        
        "env_vars": {
            "CUDA_VISIBLE_DEVICES": "0",
            "VLLM_ATTENTION_BACKEND": "FLASHINFER",
            "HF_HOME": hf_home
        }
    }

## Testing

In [ ]:
import json

metadata = get_plugin_metadata()
print(json.dumps(metadata, indent=2))

{
  "name": "cjm-transcription-plugin-voxtral-vllm",
  "version": "0.0.13",
  "type": "transcription",
  "category": "transcription",
  "interface": "cjm_transcription_plugin_system.plugin_interface.TranscriptionPlugin",
  "module": "cjm_transcription_plugin_voxtral_vllm.plugin",
  "class": "VoxtralVLLMPlugin",
  "python_path": "/home/innom-dt/miniforge3/envs/cjm-transcription-plugin-voxtral-vllm/bin/python3.12",
  "db_path": "/home/innom-dt/miniforge3/envs/cjm-transcription-plugin-voxtral-vllm/data/voxtral_vllm_transcriptions.db",
  "resources": {
    "requires_gpu": true,
    "min_gpu_vram_mb": 8192,
    "recommended_gpu_vram_mb": 16384,
    "min_system_ram_mb": 16384
  },
  "env_vars": {
    "CUDA_VISIBLE_DEVICES": "0",
    "VLLM_ATTENTION_BACKEND": "FLASHINFER",
    "HF_HOME": "/home/innom-dt/miniforge3/envs/cjm-transcription-plugin-voxtral-vllm/.cache/huggingface"
  }
}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()